# Планирование экспериментов

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#&quot;Проклятие-размерности&quot;-и-как-его-избежать" data-toc-modified-id="&quot;Проклятие-размерности&quot;-и-как-его-избежать-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>"Проклятие размерности" и как его избежать</a></span></li><li><span><a href="#Физические-и-численные-эксперименты" data-toc-modified-id="Физические-и-численные-эксперименты-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Физические и численные эксперименты</a></span></li><li><span><a href="#Проектирование-предварительных-экспериментов-(скрининг)" data-toc-modified-id="Проектирование-предварительных-экспериментов-(скрининг)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проектирование предварительных экспериментов (скрининг)</a></span><ul class="toc-item"><li><span><a href="#Оценка-распределения-элементарных-эффектов" data-toc-modified-id="Оценка-распределения-элементарных-эффектов-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Оценка распределения элементарных эффектов</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Задача инженерного дизайна: построить "суррогатную" модель _f_, которая легко оценивается и имитирует сложное поведение неизвестного объекта __f__

Обозначим $D$ (подмножество k-мерного R-пространства, $D \subseteq R^k$) - так называемую область проектирования, в которой функция f(x) непрерывна. Но мы можем получать только дискретные значения функции f {$x_i$ → $y_i$ = f($x_i$)|i = 1..n}. Это довольно дорогостоящая процедура. Так что наша задача состоит в том, чтобы используя некоторый набор этих дискретных данных (__выборку__), сконструировать приближённую функцию _f_, с помощью которой можно легко (с малыми затратами по времени и памяти) производить предсказания значения функции f для любого __x__ их $D$.

$D$ - область проектирования  
__x__ - проектные переменные  
__f__ - целевая функция  
_f_   - суррогатная модель.  


Большая часть этой книги - способы конструирования _f_ по заданной выборке. За исключением нескольких случаев, для которых уже найдены точные математические решения...  
Некоторым моделям необходимо минимальное число элементов в выборке _n_...

Важно сказать, что сконструированные таким образом функции _f_ по заданной выборке не всегда хорошо обобщаются на всю область $D$. Всё зависит от пришедшей на вход выборки.  
Получение этой выборки не всегда зависит от нас: её могли получить исследуя другие вопросы. Однако далее будет описано, как в случае возможности создания своей собственной выборки сделать такую выборку, которая позволит создать хорошо обобщающую данные суррогатную модель _f_.

## "Проклятие размерности" и как его избежать

Очевидно, чем больше в задаче переменных, тем сложнее будет функция измерения, с помощью которой мы хотим достаточно точно определить местоположение объекта. Заметим, что если некоторый уровень точности определения положения в одномерном пространстве достигается взятием выборки из _n_ позиций, то в _k_-мерном пространстве для достижения такой же точности необходимо взять выборку из _n$^k$_ точек. Это и есть "проклятие размерности". Поэтому важно с самого начала свести количество _проектных переменных k_ к минимуму.  
Задача состоит в том, чтобы среди всех переменных выделить существенные. Но сначала нам нужно сделать несколько общих замечаний о физических и вычислительных экспериментах, двух источниках, которые могут быть использованы для получения целевой функции

## Физические и численные эксперименты

Существуют 3 источника ошибок, из-за которых результаты эксперимента отклоняются от теоретических расчётов:

* __человеческая ошибка__ - осуществляется самим экспериментатором;  
* __систематическая ошибка__ - появляется из-за недостаточного соответствия между теорией и экспериментом;
* __случайная ошибка__ - появляется из-за наличия погрешности измерений.

Главным отличием между последними двумя пунктами является повторяемость. Систематическая составляющая вносит в экспериментальную ошибку всегда одно и то же значение, а случайная - каждый раз будет отличаться, и при большом числе экспериментов она будет принимать как положительные, так и отрицательные значения.  
Численные эксперименты (эксперименты, смоделированные на ЭВМ) также подвержены экспериментальной ошибке, возникающей в результате:

* __человеческая ошибка__ - 'баги' и ошибки в написанном коде, неверное введённые граничные условия при решении ДУЧП (дифференциального уравнения в частных производных) и т.д.;
* __систематическая ошибка__ - например, математическая модель невязкого обтекания тела (аппроксимация, иногда используемая для экономии вычислительного времени) будет недооценивать силы сопротивления, действующие на тело (потому что в течении на самом деле присутствует вязкость). Другой пример - ошибка, вызванная конечностью разрешения численного моделирования (например, ошибка, вызванная малым числом узлов расчётной сетки). Хоть этот тип ошибки может как переоценить значение некоторой величины, так и недооценить, но при повторении эксперимента он сделает точно также.  

Таким образом, разница между физическими и численными экспериментами заключается в том, что на вторые не влияет случайная ошибка - численные эксперименты ___детерминированы___  

__Небольшое замечание:__  
> Физические экспериментаторы часто используют слово *шум*, имея в виду случайную ошибку. Однако в некоторой литературе по численным экспериментам также встречается слово *шум*, которое должно относится к систематическим ошибкам, потому что там нет случайных.  
Чтобы не запутаться, в дальнейшем, когда мы будем говорить об ошибках физических экспериментов, будем писать слово *шум* без кавычек,а когда об ошибках численных экспериментов - будем писать "шум" - в кавычках
>  



## Проектирование предварительных экспериментов (скрининг)

Ранее мы увидели, насколько важно минимизировать число проектных переменных $x_1, x_2, ..., x_k$ перед тем, как мы попытаемся смоделировать целевую функцию __f__. Но как нам добиться такого ___скрининга___ (так мы будем называть этот процесс в дальнейшем), чтобы наш анализ всё ещё имел смысл?  
Дифференцируемость функции __f__ в области проектирования $D$ по каждой переменной: $$\frac{\partial f}{\partial x_i}|_x$$ является важным критерием для классификации:
* если $\frac{\partial f}{\partial x_i}|_x = 0, \forall x \in D$, то переменной x_i можно смело пренебречь,
* если $\frac{\partial f}{\partial x_i}|_x = const \neq 0, \forall x \in D$, то влияние переменной $x_i$ линейно и аддитивно,
* если $\frac{\partial f}{\partial x_i}|_x = g(x_i) \neq const, \forall x \in D$, то $f$ нелинейна по $x_i$,
*  если $\frac{\partial f}{\partial x_i}|_x = g(x_i),\forall x \in D, где\, g(x_i, x_j, ...) \neq const$, то $f$ нелинейна по $x_i$ и участвует во взаимодействиях с $x_j, ...$  

Приведенная выше классификация является всего лишь терминологической формулировкой, т.к. на практике мы не можем измерить $\frac{\partial f}{\partial x_i}|_x$ во всей области проектирования. Скрининговое исследование - трудная и ресурсоёмкая задача. Однако нет жёсткого правила относительно того, сколько времени должно быть потрачено на скринниг переменных, т.к. это в значительной степени зависит от задачи. Если мы ожидаем, что многими переменными можно будет пренебречь, то тщательное скрининговое исследование может значительно повысить точность последующей "суррогатной" модели. Однако, если есть (инженерные) основания, полагать, что большинство переменных имеют значительное влияние на целевую функцию, то целесообразно сосредоточить усилия на самом моделировании.  
О методах скрининга входных переменных написано много статей (Jones et al., 1998). Их принципы работы варьируются в зависимости от допущений, которые они делают относительно целевой функции и переменных. Здесь мы остановимся на алгоритме, который в 1991 году описал Моррис. Почему мы выбрали именно этот алгоритм? Потому что единственное допущение, которое он делает, заключается в том, что целевая функция детерминирована (это общая особенность для большинства вычислительных моделей).

### Оценка распределения элементарных эффектов